In [3]:
import numpy as np 
import numba 

In [7]:
def binary_lattice(lattice, rho):
        """ Generate binary lattice from the continuous fraction Brownian motion lattice 
            ρ ∈ [0,1] determines the occupancy, where ρN is the (integer) number of sites
            available to be occupied by resources
        """
        shifted_lattice = lattice + abs(np.min(lattice))    # Shift
        sorted_lattice = np.sort(shifted_lattice.flatten()) # Sort 
        # Determine cutoff point
        cutoff = sorted_lattice[int((1-rho)*lattice.shape[0]*lattice.shape[1])]
        # Generate binary lattice
        _lattice = shifted_lattice / cutoff                 # Normalize lattice
        _lattice[_lattice >= 1] = 1                         # All above cutoff to 1
        _lattice[_lattice < 1] = 0                          # All below to 0
        return np.asarray(_lattice, dtype=np.int64)

In [14]:
_lattice = np.random.random((512,512))
lattice = binary_lattice(_lattice, 0.5)
occupied = np.where(lattice!=0)
_occupied_sites = occupied[0]*512 + occupied[1]

In [9]:
@numba.jit(nopython=True, cache=True)
def test(lattice, L):    
    _occupied_sites = np.where(lattice!=0)
    occupied_sites_1D = _occupied_sites[0]*L + _occupied_sites[1]
    K = len(occupied_sites_1D)

In [10]:
%timeit test(lattice, lattice.shape[0])

2.05 ms ± 176 µs per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [15]:
@numba.jit(nopython=True, cache=True)
def test(_occupied_sites):    
    _occupied_sites = np.append(_occupied_sites, 0) 

In [16]:
%timeit test(_occupied_sites)

100 µs ± 56.8 µs per loop (mean ± std. dev. of 7 runs, 1 loop each)
